In [74]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
from splinter import Browser
import pandas as pd
import tweepy
import re

### NASA news
retrieve main headline and text from NASA news site

In [7]:
#retrieve base html
url = 'https://mars.nasa.gov/news/'
browser = Browser('chrome', headless=False)
browser.visit(url)
news_soup = bs(browser.html, "html5lib") #lxml not working in new env

In [8]:
#find first article
article_soup = news_soup.find('div', class_ = 'list_text')
article_soup

<div class="list_text"><div class="list_date">March 14, 2018</div><div class="content_title"><a href="/news/8319/nasa-mars-mission-tours-california/" target="_self">NASA Mars Mission Tours California</a></div><div class="article_teaser_body">Scientists and engineers with NASA's next mission to Mars will be touring California cities starting this month.</div></div>

In [9]:
#subsearch title and text from within article to make sure that they match
news_title = article_soup.find('div', class_ = 'content_title').text
#cleanup
news_title = news_title.replace('\n','')
news_title

'NASA Mars Mission Tours California'

In [10]:
news_p = article_soup.find('div', class_ = 'article_teaser_body').text
news_p

"Scientists and engineers with NASA's next mission to Mars will be touring California cities starting this month."

### JPL Featured Image
retrieve featured Mars image from JPL site

In [11]:
url = 'https://www.jpl.nasa.gov/spaceimages/'
browser.visit(url)

In [12]:
browser.find_by_id('menu_button').click()
#then would search in field for mars may fill in later

In [13]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [15]:
#browser.find_by_id('full_image').click()
image_soup = bs(browser.html, 'html5lib')
image_url = image_soup.find(class_ = 'carousel_item')['style']
image_url

"background-image: url('/spaceimages/images/wallpaper/PIA18886-1920x1200.jpg');"

In [16]:
featured_image_url = 'https://www.jpl.nasa.gov/' + image_url.split("'")[1]
featured_image_url

'https://www.jpl.nasa.gov//spaceimages/images/wallpaper/PIA18886-1920x1200.jpg'

### Mars Weather
scraped from nasa twitter

In [56]:
url = "https://twitter.com/marswxreport"
browser.visit(url)

In [57]:
twit_soup = bs(browser.html, 'html5lib')
tweets_html = twit_soup.findAll(class_ = "js-tweet-text-container")

In [58]:
tweets = []
dates = []
for tweet in tweets_html:
    text = tweet.find('p').text
    if 'sol' in text.lower() and 'low' in text.lower():
        tweets.append(text)
        #text.split(' ')[1]
mars_weather = tweets[0]
mars_weather

'Sol 1987 (March 09, 2018), Sunny, high -11C/12F, low -76C/-104F, pressure at 7.23 hPa, daylight 05:35-17:24'

### Mars Facts
from spacefacts.com

In [70]:
url = 'https://space-facts.com/mars/'
dfs = pd.read_html(url)

In [71]:
df = dfs[0]
df = df.rename(columns = {0:'Mars Data', 1:' '})
df = df.set_index('Mars Data')
df

,Mars Data
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [107]:
table_html = df.to_html()
re.match("",table_html)
table_html = table_html.replace('dataframe', '')
table_html = table_html.replace('<th></th>', ' ')


table_html
#search = re.search
border_pat = re.compile(r'border="."')
class_pat = re.compile(r'class=""')
table_html = table_html.replace(border_pat.search(table_html).group(), '')
table_html = table_html.replace(class_pat.search(table_html).group(), '')
table_html

'<table  >\n  <thead>\n    <tr style="text-align: right;">\n       \n      <th>Mars Data</th>\n    </tr>\n    <tr>\n       \n       \n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

### Hemisphere Images
scraped from astrogeology.usgs.gov

In [20]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [21]:
hemispheres_soup = bs(browser.html, 'html5lib').find_all(class_ = 'item')
hemispheres_soup[0]

<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>

In [23]:
base_url = 'https://astrogeology.usgs.gov'
hemisphere_imgs = {}
for hsoup in hemispheres_soup:
    sub_url = hsoup.find('a')['href']
    browser.visit(base_url + sub_url)
    soup = bs(browser.html, 'html5lib')
    for href in soup.find_all('a'):
        if 'full.jpg' in href['href']:
            print(href['href'])
            hemisphere = href['href'].split('Viking/')[1] .split('/')[0].replace('_enhanced.tif','')
            hemisphere_imgs[hemisphere] = href['href']

http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [24]:
hemisphere_imgs

{'cerberus': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'schiaparelli': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'syrtis_major': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'valles_marineris': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}